In [ ]:
import os
import re
from six.moves import urllib
import matplotlib.pyplot as plt
import mediapy as media
import numpy as np
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

from official.vision.beta.configs import video_classification
from official.vision.beta.projects.movinet.configs import movinet as movinet_configs
from official.vision.beta.projects.movinet.modeling import movinet
from official.vision.beta.projects.movinet.modeling import movinet_layers
from official.vision.beta.projects.movinet.modeling import movinet_model

tf.config.list_physical_devices('GPU')

In [ ]:
movinet_a2_hub_url = 'https://tfhub.dev/tensorflow/movinet/a2/base/kinetics-600/classification/1'

inputs = tf.keras.layers.Input(
    shape=[None, None, None, 3],
    dtype=tf.float32)

encoder = hub.KerasLayer(movinet_a2_hub_url, trainable=True)

# Important: To use tf.nn.conv3d on CPU, we must compile with tf.function.
encoder.call = tf.function(encoder.call, experimental_compile=True)

# [batch_size, 600]
outputs = encoder(dict(image=inputs))

model = tf.keras.Model(inputs, outputs)

In [ ]:
image_url = 'https://upload.wikimedia.org/wikipedia/commons/8/84/Ski_Famille_-_Family_Ski_Holidays.jpg'
image_height = 224
image_width = 224

with urllib.request.urlopen(image_url) as f:
  image = Image.open(f).resize((image_height, image_width))
video = tf.reshape(np.array(image), [1, 1, image_height, image_width, 3])
video = tf.cast(video, tf.float32) / 255.

image

In [ ]:
output = model(video)
output_label_index = tf.argmax(output, -1)[0].numpy()

print(output_label_index)

In [ ]:
num_classes = 101
num_examples = {'train':9537 , 'test':3783} #from tensorflow site

In [ ]:
batch_size = 8
num_frames = 8
frame_stride = 10
resolution = 172

def format_features(features):
  video = features['video']
  video = video[:, ::frame_stride]
  video = video[:, :num_frames]

  video = tf.reshape(video, [-1, video.shape[2], video.shape[3], 3])
  video = tf.image.resize(video, (resolution, resolution))
  video = tf.reshape(video, [-1, num_frames, resolution, resolution, 3])
  video = tf.cast(video, tf.float32) / 255.

  label = tf.one_hot(features['label'], num_classes)
  return (video, label)

train_dataset = tfds.load(
    'ucf101', 
    data_dir ='/home/evan/Datasets/tensorflow', 
    split='train',
    batch_size=batch_size,
    shuffle_files=True)
train_dataset = train_dataset.map(
    format_features,
    num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.prefetch(2)

test_dataset = tfds.load(
    'ucf101', 
    data_dir ='/home/evan/Datasets/tensorflow', 
    split='train',
    batch_size=batch_size)
test_dataset = test_dataset.map(
    format_features,
    num_parallel_calls=tf.data.AUTOTUNE,
    deterministic=True)
test_dataset = test_dataset.prefetch(2)

In [ ]:
videos, labels = next(iter(train_dataset))
media.show_videos(videos.numpy(), codec='gif', fps=5)

In [ ]:
model_id = 'a0'

tf.keras.backend.clear_session()

backbone = movinet.Movinet(
    model_id=model_id)
model = movinet_model.MovinetClassifier(
    backbone=backbone,
    num_classes=600)
model.build([batch_size, num_frames, resolution, resolution, 3])

# Load pretrained weights from TF Hub
movinet_hub_url = f'https://tfhub.dev/tensorflow/movinet/{model_id}/base/kinetics-600/classification/1'
movinet_hub_model = hub.KerasLayer(movinet_hub_url, trainable=True)
pretrained_weights = {w.name: w for w in movinet_hub_model.weights}
model_weights = {w.name: w for w in model.weights}

  #rename weights appropriately
for name in model_weights:
  pretrained_name = re.sub(r'block(\d)_', r'b\1/', name)
  pretrained_name = re.sub(r'layer(\d)', r'l\1', pretrained_name)
  model_weights[name].assign(pretrained_weights[pretrained_name])

# Wrap the backbone with a new classifier to create a new classifier head
# with num_classes outputs
model = movinet_model.MovinetClassifier(
    backbone=backbone,
    num_classes=num_classes)
model.build([batch_size, num_frames, resolution, resolution, 3])

# Freeze all layers except for the final classifier head
for layer in model.layers[:-1]:
  layer.trainable = False
model.layers[-1].trainable = True

In [ ]:
num_epochs = 3

train_steps = num_examples['train'] // batch_size
total_train_steps = train_steps * num_epochs
test_steps = num_examples['test'] // batch_size

loss_obj = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True,
    label_smoothing=0.1)

metrics = [
    tf.keras.metrics.TopKCategoricalAccuracy(
        k=1, name='top_1', dtype=tf.float32),
    tf.keras.metrics.TopKCategoricalAccuracy(
        k=5, name='top_5', dtype=tf.float32),
]

initial_learning_rate = 0.01
learning_rate = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps=total_train_steps,
)
optimizer = tf.keras.optimizers.RMSprop(
    learning_rate, rho=0.9, momentum=0.9, epsilon=1.0, clipnorm=1.0)

model.compile(loss=loss_obj, optimizer=optimizer, metrics=metrics)

callbacks = [
    tf.keras.callbacks.TensorBoard(),
]

In [ ]:
results = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=num_epochs,
    steps_per_epoch=train_steps,
    validation_steps=test_steps,
    callbacks=callbacks,
    validation_freq=1,
    verbose=1)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs --port 0